In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 45.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=a0e1695250cc17054600d360ae64b85efe7cbc76c7d469a4e737be0b0b82e04d
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Classifier").getOrCreate()

In [8]:
from pyspark.ml.linalg import Vectors

In [17]:
data = [(Vectors.dense([0.0, 0.0]), 2.0), (Vectors.dense([1.0, 1.0]), 2.0),
        (Vectors.dense([9.0, 8.0]), 2.0), (Vectors.dense([8.0, 9.0]), 2.0)]
data

[(DenseVector([0.0, 0.0]), 2.0),
 (DenseVector([1.0, 1.0]), 2.0),
 (DenseVector([9.0, 8.0]), 2.0),
 (DenseVector([8.0, 9.0]), 2.0)]

In [10]:
 from pyspark.ml.clustering import KMeans

In [16]:
df = spark.createDataFrame(data, ["features", "weighCol"])
df

DataFrame[features: vector, weighCol: double]

In [14]:
kmeans = KMeans(k=2)
kmeans

KMeans_4cdcf6a4d565

In [15]:
kmeans.setSeed(1)
kmeans

KMeans_4cdcf6a4d565

In [18]:
kmeans.setWeightCol("weighCol")
kmeans

KMeans_4cdcf6a4d565

In [19]:
kmeans.setMaxIter(10)

KMeans_4cdcf6a4d565

In [20]:
kmeans.getMaxIter()

10

In [22]:
kmeans.clear(kmeans.maxIter)

In [24]:
model = kmeans.fit(df)
model

KMeansModel: uid=KMeans_4cdcf6a4d565, k=2, distanceMeasure=euclidean, numFeatures=2

In [25]:
model.getDistanceMeasure()

'euclidean'

In [26]:
model.setPredictionCol("newPrediction")

KMeansModel: uid=KMeans_4cdcf6a4d565, k=2, distanceMeasure=euclidean, numFeatures=2

In [27]:
model.predict(df.head().features)

0

In [29]:
centers = model.clusterCenters()
centers

[array([0.5, 0.5]), array([8.5, 8.5])]

In [30]:
len(centers)

2

In [32]:
transformed = model.transform(df).select("features", "newPrediction")
transformed

DataFrame[features: vector, newPrediction: int]

In [34]:
rows = transformed.collect()
rows

[Row(features=DenseVector([0.0, 0.0]), newPrediction=0),
 Row(features=DenseVector([1.0, 1.0]), newPrediction=0),
 Row(features=DenseVector([9.0, 8.0]), newPrediction=1),
 Row(features=DenseVector([8.0, 9.0]), newPrediction=1)]

In [35]:
rows[0].newPrediction == rows[1].newPrediction

True

In [36]:
rows[2].newPrediction == rows[3].newPrediction

True

In [37]:
model.hasSummary

True

In [38]:
summary = model.summary

In [39]:
summary.k

2

In [40]:
summary.clusterSizes

[2, 2]

In [41]:
summary.trainingCost

4.0

In [43]:
temp_path = "/content/sample_data/"
kmeans_path = temp_path + "mykmeans"

In [44]:
kmeans.save(kmeans_path)
kmeans2 = KMeans.load(kmeans_path)
kmeans2.getK()

2

In [48]:
from pyspark.ml.clustering import KMeansModel

In [50]:
model_path = temp_path +"/kmeans_model1"
model.save(model_path)
model2 = KMeansModel.load(model_path)
model2.hasSummary

False

In [53]:
model.clusterCenters()[0] == model2.clusterCenters()[0]

array([ True,  True])

In [54]:
model.clusterCenters()[1] == model2.clusterCenters()[1]

array([ True,  True])

In [55]:
model.transform(df).take(1) == model2.transform(df).take(1)

True